In [ ]:
import os
import sys

os.environ["SPARK_HOME"] = "/home/talentum/spark"
os.environ["PYLIB"] = os.environ["SPARK_HOME"] + "/python/lib"
# In below two lines, use /usr/bin/python2.7 if you want to use Python 2
os.environ["PYSPARK_PYTHON"] = "/usr/bin/python3.6" 
os.environ["PYSPARK_DRIVER_PYTHON"] = "/usr/bin/python3"
sys.path.insert(0, os.environ["PYLIB"] +"/py4j-0.10.7-src.zip")
sys.path.insert(0, os.environ["PYLIB"] +"/pyspark.zip")

# NOTE: Whichever package you want mention here.
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0 pyspark-shell' 
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-avro_2.11:2.4.0 pyspark-shell'
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0,org.apache.spark:spark-avro_2.11:2.4.3 pyspark-shell'
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0,org.apache.spark:spark-avro_2.11:2.4.0 pyspark-shell'

In [ ]:
#Entrypoint 2.x
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Project").enableHiveSupport().getOrCreate()

# On yarn:
# spark = SparkSession.builder.appName("Spark SQL basic example").enableHiveSupport().master("yarn").getOrCreate()
# specify .master("yarn")

sc = spark.sparkContext

In [1]:
# !pip install pyspark

In [2]:
# # Ext. code 
# import pyspark
# from pyspark.sql import SparkSession
# ss = SparkSession.builder.appName('Project').getOrCreate()
# ss

In [3]:
from pyspark.sql.types import StructType,StructField,StringType,IntegerType,DoubleType,TimestampType

In [4]:
from pyspark.sql.functions import col, sum, round ,split

In [6]:
df1 = spark.read.csv("price_data.csv",header=True, inferSchema=True)

In [7]:
df1.printSchema()

root
 |-- baseFare: integer (nullable = true)
 |-- reservationCharge: integer (nullable = true)
 |-- superfastCharge: integer (nullable = true)
 |-- fuelAmount: double (nullable = true)
 |-- totalConcession: integer (nullable = true)
 |-- tatkalFare: integer (nullable = true)
 |-- serviceTax: double (nullable = true)
 |-- otherCharge: integer (nullable = true)
 |-- cateringCharge: integer (nullable = true)
 |-- dynamicFare: integer (nullable = true)
 |-- totalFare: integer (nullable = true)
 |-- availability: string (nullable = true)
 |-- trainNumber: integer (nullable = true)
 |-- timeStamp: timestamp (nullable = true)
 |-- fromStnCode: string (nullable = true)
 |-- toStnCode: string (nullable = true)
 |-- classCode: string (nullable = true)
 |-- distance: integer (nullable = true)
 |-- duration: double (nullable = true)



In [9]:
df1.columns

['baseFare',
 'reservationCharge',
 'superfastCharge',
 'fuelAmount',
 'totalConcession',
 'tatkalFare',
 'serviceTax',
 'otherCharge',
 'cateringCharge',
 'dynamicFare',
 'totalFare',
 'availability',
 'trainNumber',
 'timeStamp',
 'fromStnCode',
 'toStnCode',
 'classCode',
 'distance',
 'duration']

In [10]:
df1=df1.drop("availability","superfastCharge","fuelAmount","totalConcession","tatkalFare","otherCharge","cateringCharge","dynamicFare")

In [11]:
df1.show()

+--------+-----------------+----------+---------+-----------+--------------------+-----------+---------+---------+--------+--------+
|baseFare|reservationCharge|serviceTax|totalFare|trainNumber|           timeStamp|fromStnCode|toStnCode|classCode|distance|duration|
+--------+-----------------+----------+---------+-----------+--------------------+-----------+---------+---------+--------+--------+
|    1059|               60|      56.0|     1175|      11464|2023-10-03 22:13:...|        JBP|     SRID|       1A|      54|    33.0|
|     626|               50|      34.0|      710|      11464|2023-10-03 22:13:...|        JBP|     SRID|       2A|      54|    33.0|
|     441|               40|      24.0|      505|      11464|2023-10-03 22:13:...|        JBP|     SRID|       3A|      54|    33.0|
|     125|               20|       0.0|      145|      11464|2023-10-03 22:13:...|        JBP|     SRID|       SL|      54|    33.0|
|    1059|               60|      56.0|     1175|      11464|2023-10-

In [12]:
df1.printSchema()

root
 |-- baseFare: integer (nullable = true)
 |-- reservationCharge: integer (nullable = true)
 |-- serviceTax: double (nullable = true)
 |-- totalFare: integer (nullable = true)
 |-- trainNumber: integer (nullable = true)
 |-- timeStamp: timestamp (nullable = true)
 |-- fromStnCode: string (nullable = true)
 |-- toStnCode: string (nullable = true)
 |-- classCode: string (nullable = true)
 |-- distance: integer (nullable = true)
 |-- duration: double (nullable = true)



In [13]:
Column = ['baseFare',
 'reservationCharge',
 'superfastCharge',
 'fuelAmount',
 'totalConcession',
 'tatkalFare',
 'serviceTax',
 'otherCharge',
 'cateringCharge',
 'dynamicFare',
 'totalFare',
 'availability',
 'trainNumber',
 'timeStamp',
 'fromStnCode',
 'toStnCode',
 'classCode',
 'distance',
 'duration']

In [14]:
new_df = df1.select([col(col_name).alias(col_name.capitalize()) for col_name in df1.columns])

In [15]:
new_df.show(4)

+--------+-----------------+----------+---------+-----------+--------------------+-----------+---------+---------+--------+--------+
|Basefare|Reservationcharge|Servicetax|Totalfare|Trainnumber|           Timestamp|Fromstncode|Tostncode|Classcode|Distance|Duration|
+--------+-----------------+----------+---------+-----------+--------------------+-----------+---------+---------+--------+--------+
|    1059|               60|      56.0|     1175|      11464|2023-10-03 22:13:...|        JBP|     SRID|       1A|      54|    33.0|
|     626|               50|      34.0|      710|      11464|2023-10-03 22:13:...|        JBP|     SRID|       2A|      54|    33.0|
|     441|               40|      24.0|      505|      11464|2023-10-03 22:13:...|        JBP|     SRID|       3A|      54|    33.0|
|     125|               20|       0.0|      145|      11464|2023-10-03 22:13:...|        JBP|     SRID|       SL|      54|    33.0|
+--------+-----------------+----------+---------+-----------+--------

In [16]:
new_df.columns

['Basefare',
 'Reservationcharge',
 'Servicetax',
 'Totalfare',
 'Trainnumber',
 'Timestamp',
 'Fromstncode',
 'Tostncode',
 'Classcode',
 'Distance',
 'Duration']

In [17]:
df_percent = new_df.withColumn("Service_tax", round(col("Servicetax") / col("Totalfare") * 100, 2))
df_percent.select('Service_tax').show()

+-----------+
|Service_tax|
+-----------+
|       4.77|
|       4.79|
|       4.75|
|        0.0|
|       4.77|
|       4.79|
|       4.75|
|        0.0|
|       4.77|
|       4.79|
|       4.75|
|        0.0|
|       4.77|
|       4.79|
|       4.75|
|        0.0|
|       4.77|
|       4.79|
|       4.75|
|        0.0|
+-----------+
only showing top 20 rows



In [18]:
df_percent.show(4)

+--------+-----------------+----------+---------+-----------+--------------------+-----------+---------+---------+--------+--------+-----------+
|Basefare|Reservationcharge|Servicetax|Totalfare|Trainnumber|           Timestamp|Fromstncode|Tostncode|Classcode|Distance|Duration|Service_tax|
+--------+-----------------+----------+---------+-----------+--------------------+-----------+---------+---------+--------+--------+-----------+
|    1059|               60|      56.0|     1175|      11464|2023-10-03 22:13:...|        JBP|     SRID|       1A|      54|    33.0|       4.77|
|     626|               50|      34.0|      710|      11464|2023-10-03 22:13:...|        JBP|     SRID|       2A|      54|    33.0|       4.79|
|     441|               40|      24.0|      505|      11464|2023-10-03 22:13:...|        JBP|     SRID|       3A|      54|    33.0|       4.75|
|     125|               20|       0.0|      145|      11464|2023-10-03 22:13:...|        JBP|     SRID|       SL|      54|    33.

In [19]:
df_percent.select('Timestamp').show()

+--------------------+
|           Timestamp|
+--------------------+
|2023-10-03 22:13:...|
|2023-10-03 22:13:...|
|2023-10-03 22:13:...|
|2023-10-03 22:13:...|
|2023-10-03 22:13:...|
|2023-10-03 22:13:...|
|2023-10-03 22:13:...|
|2023-10-03 22:13:...|
|2023-10-03 22:13:...|
|2023-10-03 22:13:...|
|2023-10-03 22:13:...|
|2023-10-03 22:13:...|
|2023-10-03 22:13:...|
|2023-10-03 22:13:...|
|2023-10-03 22:13:...|
|2023-10-03 22:13:...|
|2023-10-03 22:13:...|
|2023-10-03 22:13:...|
|2023-10-03 22:13:...|
|2023-10-03 22:13:...|
+--------------------+
only showing top 20 rows



In [20]:
df_split = df_percent.withColumn("date", split(col("Timestamp"), " ")[0]) \
                .withColumn("time", split(col("Timestamp"), " ")[1])

In [21]:
df_split.show(4)

+--------+-----------------+----------+---------+-----------+--------------------+-----------+---------+---------+--------+--------+-----------+----------+---------------+
|Basefare|Reservationcharge|Servicetax|Totalfare|Trainnumber|           Timestamp|Fromstncode|Tostncode|Classcode|Distance|Duration|Service_tax|      date|           time|
+--------+-----------------+----------+---------+-----------+--------------------+-----------+---------+---------+--------+--------+-----------+----------+---------------+
|    1059|               60|      56.0|     1175|      11464|2023-10-03 22:13:...|        JBP|     SRID|       1A|      54|    33.0|       4.77|2023-10-03|22:13:07.781307|
|     626|               50|      34.0|      710|      11464|2023-10-03 22:13:...|        JBP|     SRID|       2A|      54|    33.0|       4.79|2023-10-03|22:13:07.781307|
|     441|               40|      24.0|      505|      11464|2023-10-03 22:13:...|        JBP|     SRID|       3A|      54|    33.0|       4

In [22]:
df_split.select("Date","Time").show()

+----------+---------------+
|      Date|           Time|
+----------+---------------+
|2023-10-03|22:13:07.781307|
|2023-10-03|22:13:07.781307|
|2023-10-03|22:13:07.781307|
|2023-10-03|22:13:07.781307|
|2023-10-03|22:13:07.781307|
|2023-10-03|22:13:07.781307|
|2023-10-03|22:13:07.781307|
|2023-10-03|22:13:07.781307|
|2023-10-03|22:13:07.781307|
|2023-10-03|22:13:07.781307|
|2023-10-03|22:13:07.781307|
|2023-10-03|22:13:07.781307|
|2023-10-03|22:13:07.781307|
|2023-10-03|22:13:07.781307|
|2023-10-03|22:13:07.781307|
|2023-10-03|22:13:07.781307|
|2023-10-03|22:13:07.781307|
|2023-10-03|22:13:07.781307|
|2023-10-03|22:13:07.781307|
|2023-10-03|22:13:07.781307|
+----------+---------------+
only showing top 20 rows



In [23]:
df_timestamp=df_split.drop("Timestamp")

In [24]:
df_timestamp.show()

+--------+-----------------+----------+---------+-----------+-----------+---------+---------+--------+--------+-----------+----------+---------------+
|Basefare|Reservationcharge|Servicetax|Totalfare|Trainnumber|Fromstncode|Tostncode|Classcode|Distance|Duration|Service_tax|      date|           time|
+--------+-----------------+----------+---------+-----------+-----------+---------+---------+--------+--------+-----------+----------+---------------+
|    1059|               60|      56.0|     1175|      11464|        JBP|     SRID|       1A|      54|    33.0|       4.77|2023-10-03|22:13:07.781307|
|     626|               50|      34.0|      710|      11464|        JBP|     SRID|       2A|      54|    33.0|       4.79|2023-10-03|22:13:07.781307|
|     441|               40|      24.0|      505|      11464|        JBP|     SRID|       3A|      54|    33.0|       4.75|2023-10-03|22:13:07.781307|
|     125|               20|       0.0|      145|      11464|        JBP|     SRID|       SL| 

In [25]:
    import pandas as pd

# Convert PySpark DataFrame to Pandas DataFrame
pandas_df = df_timestamp.toPandas()

In [27]:
# Specify the output path
output_path = "Fare.csv"

# Write Pandas DataFrame to CSV file with header
pandas_df.to_csv(output_path, header=True, index=False)

In [28]:
df_timestamp.printSchema()

root
 |-- Basefare: integer (nullable = true)
 |-- Reservationcharge: integer (nullable = true)
 |-- Servicetax: double (nullable = true)
 |-- Totalfare: integer (nullable = true)
 |-- Trainnumber: integer (nullable = true)
 |-- Fromstncode: string (nullable = true)
 |-- Tostncode: string (nullable = true)
 |-- Classcode: string (nullable = true)
 |-- Distance: integer (nullable = true)
 |-- Duration: double (nullable = true)
 |-- Service_tax: double (nullable = true)
 |-- date: string (nullable = true)
 |-- time: string (nullable = true)



In [29]:
# url = "jdbc:mysql://127.0.0.1:3306/test?useSSL=false&allowPublicKeyRetrieval=true"
# driver = "com.mysql.jdbc.Driver"
# user = "bigdata"
# password = "Bigdata@123"

In [30]:
mysql_url = "jdbc:mysql://127.0.0.1:3306/project?useSSL=false&allowPublicKeyRetrieval=true"
conn_prop={
    
    "user":"bigdata",
    "password":"Bigdata@123",
    "driver":"com.mysql.jdbc.Driver"
}

table_name="train_fare"

In [ ]:
df_timestamp.write.jdbc(url=mysql_url,table=table_name,mode="overwrite",properties=conn_prop)